# Optimize Plant

Once performance curves are available for chillers it is possible to look at how to run a plant (a collection of chillers) at the lowest kW (Cost).  Given a collection of chillers, a given Delta T Lift and total tons, the code provides the Load (which can be converted to tons) for each chiller for optimal performance. 

Optimization done with scipy optimum.minimum function.

In [1]:
import pickle

from scipy import optimize
import numpy as np
import pandas as pd

## Only B Plant
all chillers 900 Tons
CH1, 2 & 3 same, Ch4 different

In [2]:
def GetkW(Tons, TotalTons, Ch_lift_lines, DesignLift):
    Load = Tons/TotalTons
    
    x_line = Ch_lift_lines[DesignLift][0]
    y_line = Ch_lift_lines[DesignLift][1]

    kW = np.interp(Load,x_line,y_line)*Tons
    
    return kW

In [3]:
def f(Ton,TotalTons,ChMaxTons,all_lift_lines,DesignLift): 
    max_Ch = len(Ton)
    kW_total = 0
    for c in range(max_Ch):
        kW = GetkW(Ton[c],ChMaxTons[c],all_lift_lines[c], DesignLift)
        kW_total += kW
        
    return kW_total

def fun(x,TotalTons):
    return TotalTons-sum(x)


In [8]:
def Optimum_Chillers(TotalTons, DesignLift, ChList, ChMaxTons):
    num_chillers = len(ChList)
    Start_Tons = []
    lift_lines = []
    for Chiller in ChList:
        Ch_lift_lines = pickle.load( open('data/'+Chiller+"lift_lines.pkl", "rb" ) )
        lift_lines.append(Ch_lift_lines)        
        Start_Tons.append(TotalTons/num_chillers)
    cons = ({'type': 'eq', 'fun': fun, 'args': [TotalTons]})
    bnds = [(0.15*ChMaxTons[i], ChMaxTons[i]) for i in range(num_chillers)]
    
    Tons = optimize.minimize(f,Start_Tons,bounds=bnds,constraints=cons,
                             args=(TotalTons,ChMaxTons,lift_lines,DesignLift)).x
    kW_total = f(Tons,TotalTons,ChMaxTons,lift_lines,DesignLift)

    print(Tons,sum(Tons))
    print(f'kW={kW_total} for chillers {ChList}:')
    for c in range(num_chillers):
        kW = GetkW(Tons[c],ChMaxTons[c],lift_lines[c], DesignLift)
        print(f'Chiller {ChList[c]} with {Tons[c]/ChMaxTons[c]} Lift and {kW/Tons[c]} kW/Ton')
#        print(Tons[c],ChMaxTons[c])
    print('')
    
    return Tons, kW_total


In [127]:
# print Lift and kW/Ton for best chiller

# best chiller combination
DesignLift = 30
TotalTons = 900

ChMaxTons = ([900, 900])
ChList = (['B1','B2'])

Tons, kW = Optimum_Chillers(TotalTons, DesignLift, ChList, ChMaxTons)

[450. 450.] 900.0
kW=408.0391323896223 for chillers ['B1', 'B2']:
Chiller B1 with 0.5 Lift and 0.453376813766247 kW/Ton
Chiller B2 with 0.5 Lift and 0.453376813766247 kW/Ton



In [128]:
ChMaxTons = ([900,900,900,900])

DesignLift = 30
TotalTons = 2000

ChList = (['B1','B2','B3','B4'])
Tons, kW = Optimum_Chillers(TotalTons, DesignLift, ChList, ChMaxTons)

ChMaxTons = ([900,900,900])
ChList = (['B1','B2','B3'])
Tons, kW = Optimum_Chillers(TotalTons, DesignLift, ChList, ChMaxTons)
ChList = (['B1','B2','B4'])
Tons, kW = Optimum_Chillers(TotalTons, DesignLift, ChList, ChMaxTons)
ChList = (['B1','B3','B4'])
Tons, kW = Optimum_Chillers(TotalTons, DesignLift, ChList, ChMaxTons)
ChList = (['B2','B3','B4'])
Tons, kW = Optimum_Chillers(TotalTons, DesignLift, ChList, ChMaxTons)

ChMaxTons = ([900,900])
ChList = (['B1','B2'])
Tons, kW = Optimum_Chillers(TotalTons, DesignLift, ChList, ChMaxTons)
ChList = (['B1','B3'])
Tons, kW = Optimum_Chillers(TotalTons, DesignLift, ChList, ChMaxTons)
ChList = (['B1','B4'])
Tons, kW = Optimum_Chillers(TotalTons, DesignLift, ChList, ChMaxTons)
ChList = (['B2','B3'])
CTons, kW = Optimum_Chillers(TotalTons, DesignLift, ChList, ChMaxTons)
ChList = (['B2','B4'])
Tons, kW = Optimum_Chillers(TotalTons, DesignLift, ChList, ChMaxTons)
ChList = (['B3','B4'])


[519.66667234 519.66665839 519.66667145 440.99999781] 1999.9999999999998
kW=864.245077972533 for chillers ['B1', 'B2', 'B3', 'B4']:
Chiller B1 with 0.5774074137083962 Lift and 0.4354599708860764 kW/Ton
Chiller B2 with 0.5774073982164762 Lift and 0.43545997364856337 kW/Ton
Chiller B3 with 0.5774074127256564 Lift and 0.4354599710613165 kW/Ton
Chiller B4 with 0.4899999975716934 Lift and 0.4203242217719321 kW/Ton

[666.66666667 666.66666667 666.66666667] 2000.0
kW=853.0012286179693 for chillers ['B1', 'B2', 'B3']:
Chiller B1 with 0.7407407407407407 Lift and 0.42650061430898467 kW/Ton
Chiller B2 with 0.7407407407407407 Lift and 0.42650061430898467 kW/Ton
Chiller B3 with 0.7407407407407407 Lift and 0.42650061430898467 kW/Ton

[707.50001767 707.50001767 584.99996466] 2000.0
kW=855.651167694249 for chillers ['B1', 'B2', 'B4']:
Chiller B1 with 0.7861111307423448 Lift and 0.4309826679060691 kW/Ton
Chiller B2 with 0.7861111307423448 Lift and 0.4309826679060691 kW/Ton
Chiller B4 with 0.64999996073

In [14]:
ChMaxTons = ([900,900,1200,1200])

DesignLift = 30
TotalTons = 2400

ChMaxTons = ([900,900,1200,1200])
ChList = (['B1','B2','C1','C2'])
Tons, kW = Optimum_Chillers(TotalTons, DesignLift, ChList, ChMaxTons)
kW_min = kW
ChList_min = ChList

ChMaxTons = ([900,900,1200])
ChList = (['B1','B2','C1'])
Tons, kW = Optimum_Chillers(TotalTons, DesignLift, ChList, ChMaxTons)
if (kW < kW_min):
    kW_min = kW
    ChList_min = ChList

ChMaxTons = ([900,900,1200])
ChList = (['B1','B2','C2'])
Tons, kW = Optimum_Chillers(TotalTons, DesignLift, ChList, ChMaxTons)
if (kW < kW_min):
    kW_min = kW
    ChList_min = ChList
ChMaxTons = ([900,1200,1200])
ChList = (['B1','C1','C2'])
Tons, kW = Optimum_Chillers(TotalTons, DesignLift, ChList, ChMaxTons)
if (kW < kW_min):
    kW_min = kW
    ChList_min = ChList
ChMaxTons = ([900,1200,1200])
ChList = (['B2','C1','C2'])
Tons, kW = Optimum_Chillers(TotalTons, DesignLift, ChList, ChMaxTons)
if (kW < kW_min):
    kW_min = kW
    ChList_min = ChList

ChMaxTons = ([900,900])
ChList = (['B1','B2'])
Tons, kW = Optimum_Chillers(TotalTons, DesignLift, ChList, ChMaxTons)
if (kW < kW_min):
    kW_min = kW
    ChList_min = ChList
ChMaxTons = ([900,1200])
ChList = (['B1','C1'])
Tons, kW = Optimum_Chillers(TotalTons, DesignLift, ChList, ChMaxTons)
if (kW < kW_min):
    kW_min = kW
    ChList_min = ChList
ChMaxTons = ([900,1200])
ChList = (['B1','C2'])
Tons, kW = Optimum_Chillers(TotalTons, DesignLift, ChList, ChMaxTons)
if (kW < kW_min):
    kW_min = kW
    ChList_min = ChList
ChMaxTons = ([900,1200])
ChList = (['B2','C1'])
CTons, kW = Optimum_Chillers(TotalTons, DesignLift, ChList, ChMaxTons)
if (kW < kW_min):
    kW_min = kW
    ChList_min = ChList
ChMaxTons = ([900,1200])
ChList = (['B2','C2'])
Tons, kW = Optimum_Chillers(TotalTons, DesignLift, ChList, ChMaxTons)
if (kW < kW_min):
    kW_min = kW
    ChList_min = ChList
ChMaxTons = ([1200,1200])
ChList = (['C1','C2'])
Tons, kW = Optimum_Chillers(TotalTons, DesignLift, ChList, ChMaxTons)
if (kW < kW_min):
    kW_min = kW
    ChList_min = ChList

print(f'For minimum kW ({kW_min}), run with',ChList_min)

[ 135.          135.         1064.94649656 1065.05350344] 2400.0
kW=915.9129687313118 for chillers ['B1', 'B2', 'C1', 'C2']:
Chiller B1 with 0.15000000000000005 Lift and 0.6508316785599737 kW/Ton
Chiller B2 with 0.15 Lift and 0.6508316785599737 kW/Ton
Chiller C1 with 0.8874554137964431 Lift and 0.3474989435990864 kW/Ton
Chiller C2 with 0.8875445862035569 Lift and 0.34751365239447957 kW/Ton

[762.00028845 762.00028845 875.99942311] 2400.0
kW=879.8746942430889 for chillers ['B1', 'B2', 'C1']:
Chiller B1 with 0.8466669871635414 Lift and 0.38092641562487073 kW/Ton
Chiller B2 with 0.8466669871635414 Lift and 0.38092641562487073 kW/Ton
Chiller C1 with 0.7299995192546881 Lift and 0.34171554133590537 kW/Ton

[762.00028845 762.00028845 875.99942311] 2400.0
kW=879.8746942430889 for chillers ['B1', 'B2', 'C2']:
Chiller B1 with 0.8466669871635414 Lift and 0.38092641562487073 kW/Ton
Chiller B2 with 0.8466669871635414 Lift and 0.38092641562487073 kW/Ton
Chiller C2 with 0.7299995192546881 Lift and 0.

In [111]:
1200 * .928333

1113.9995999999999

In [17]:
Tons = [600,600,600,600]
ChList = (['B1','B2','C1','C2'])
ChMaxTons = ([900,900,1200,1200])

lift_lines = []
for Chiller in ChList:
    Ch_lift_lines = pickle.load( open('data/'+Chiller+"lift_lines.pkl", "rb" ) )
    lift_lines.append(Ch_lift_lines)        

kW_total = 0
for c in range(len(ChList)):
    kW = GetkW(Tons[c],ChMaxTons[c],lift_lines[c], DesignLift)
    kW_total += kW
    
print(kW_total)

print(f'cost per day before ${(kW_total) * 24 * 0.10}')
print(f'cost per day after  ${(kW_min) * 24 * 0.10}')
print(f'saving per day      ${(kW_total - kW_min) * 24 * 0.10} ({((kW_total - kW_min)/kW_min)*100}%)\n')
print(f'cost per year before ${(kW_total) * 24 * 0.10* 365}')
print(f'cost per year after  ${(kW_min) * 24 * 0.10 * 365}')
print(f'saving per year      ${(kW_total - kW_min) * 24 * 0.10 * 365} ({((kW_total - kW_min)/kW_min)*100}%)')

992.6621940968209
cost per day before $2382.3892658323707
cost per day after  $2062.9575777153304
saving per day      $319.43168811704004 (15.484161747562544%)

cost per year before $869572.0820288153
cost per year after  $752979.5158660957
saving per year      $116592.56616271961 (15.484161747562544%)


In [16]:
TotalTons = 2400

Tons = [2400/3,2400/3,2400/3]
ChList = (['B2','C1','C2'])
ChMaxTons = ([900,1200,1200])

lift_lines = []
for Chiller in ChList:
    Ch_lift_lines = pickle.load( open('data/'+Chiller+"lift_lines.pkl", "rb" ) )
    lift_lines.append(Ch_lift_lines)        

kW_total = 0
for c in range(len(ChList)):
    kW = GetkW(Tons[c],ChMaxTons[c],lift_lines[c], DesignLift)
    kW_total += kW
    
print(kW_total)

print(f'cost per day before ${(kW_total) * 24 * 0.10}')
print(f'cost per day after  ${(kW_min) * 24 * 0.10}')
print(f'saving per day      ${(kW_total - kW_min) * 24 * 0.10} ({((kW_total - kW_min)/kW_min)*100}%)\n')
print(f'cost per year before ${(kW_total) * 24 * 0.10* 365}')
print(f'cost per year after  ${(kW_min) * 24 * 0.10 * 365}')
print(f'saving per year      ${(kW_total - kW_min) * 24 * 0.10 * 365} ({((kW_total - kW_min)/kW_min)*100}%)')

862.7960863927887
cost per day before $2070.7106073426926
cost per day after  $2062.9575777153304
saving per day      $7.753029627362503 (0.3758210886696358%)

cost per year before $755809.3716800828
cost per year after  $752979.5158660957
saving per year      $2829.8558139873135 (0.3758210886696358%)
